In [1]:
filename = "rdm.png"

In [2]:
import cv2
import numpy as np


def segment_fingerprint(image_path):
    # Load the image
    img = cv2.imread(image_path)
    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    # Apply Otsu's thresholding to binarize the image
    _, thresholded = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    # Find the contours in the binary image
    contours, _ = cv2.findContours(thresholded, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # Select the contour with the largest area as the fingerprint region
    fingerprint_contour = max(contours, key=cv2.contourArea)
    # Create a mask from the fingerprint contour
    mask = np.zeros(gray.shape, dtype=np.uint8)
    cv2.drawContours(mask, [fingerprint_contour], 0, 255, -1)
    # Apply the mask to the original image to extract the fingerprint region
    fingerprint = cv2.bitwise_and(img, img, mask=mask)
    # Convert the fingerprint region to grayscale
    fingerprint_gray = cv2.cvtColor(fingerprint, cv2.COLOR_BGR2GRAY)
    # Return the extracted fingerprint
    
    cv2.imwrite(f"pre/{filename}" , fingerprint_gray)
    return fingerprint_gray

pre = segment_fingerprint(f"contact-pre/{filename}")


In [3]:
def highlight_cretases(fingerprint):
    # Apply a Gabor filter to the fingerprint to enhance the ridges
    ksize = 1
    sigma = 2.0
    theta = np.pi / 4
    lambd = 10.0
    gamma = 0.5
    gabor_kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, 0, cv2.CV_32F)
    filtered_fingerprint = cv2.filter2D(fingerprint, cv2.CV_8UC3, gabor_kernel)
    # Apply adaptive thresholding to binarize the filtered image
    binarized_fingerprint = cv2.adaptiveThreshold(filtered_fingerprint, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 2)
    # Return the binarized fingerprint
    cv2.imwrite(f"post/{filename}" , binarized_fingerprint)
    return binarized_fingerprint
post = highlight_cretases(pre)

In [4]:
post.shape

(232, 236)

In [8]:
def split_image(image):
    # Get the dimensions of the input image
    height, width = image.shape
    # Calculate the size of the center region
    center_size = 40
    # Calculate the position of the center region
    center_x = int(width / 2 - center_size  ) 
    center_y = int(height / 2 - center_size  )

    # Create an empty list to hold the sub-images
    sub_images = []
    n = 2
    # Loop over the rows and columns
    for i in range(5):
        for j in range(3):
            # Calculate the position of the sub-image
            x = center_x + j * center_size
            y = center_y + i * center_size
            if x >= width or y>=height:
                break
            # Extract the sub-image from the input image
            sub_image = image[y:y+center_size, x:x+center_size]
            # Add the sub-image to the list
            sub_images.append(sub_image)
    # Return the list of sub-images
    for i in range(len(sub_images)):
        cv2.imwrite('contact-saved/' + str(i) + '.png' , sub_images[i])
    return sub_images

split_image(post)

[array([[  0,   0,   0, ...,   0,   0,   0],
        [  0,   0, 255, ...,   0,   0,   0],
        [  0, 255, 255, ...,   0, 255,   0],
        ...,
        [255,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0, 255, 255],
        [  0,   0, 255, ...,   0, 255, 255]], dtype=uint8),
 array([[  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ..., 255,   0,   0],
        [  0, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255,   0],
        [255, 255,   0, ..., 255, 255, 255],
        [255,   0,   0, ..., 255, 255, 255]], dtype=uint8),
 array([[  0,   0, 255, ..., 255, 255,   0],
        [  0,   0, 255, ..., 255, 255,   0],
        [  0,   0,   0, ..., 255, 255, 255],
        ...,
        [  0,   0, 255, ...,   0,   0, 255],
        [255,   0,   0, ...,   0,   0, 255],
        [255,   0,   0, ...,   0,   0, 255]], dtype=uint8),
 array([[  0, 255, 255, ..., 255, 255, 255],
        [  0, 255, 255, ..., 255, 255, 255],
        [255, 25